In [1]:
#import required libraries
!pip install tensorflow
import numpy as np
import pandas as pd
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

2024-04-14 03:14:50.758905: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-14 03:14:50.763746: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-14 03:14:50.828517: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
jokes_df = pd.read_csv('shortjokes.csv')
jokes_df.head()

,ID,Joke
0,1,"[me narrating a documentary about narrators] ""..."
1,2,Telling my daughter garlic is good for you. Go...
2,3,I've been going through a really rough period ...
3,4,"If I could have dinner with anyone, dead or al..."
4,5,Two guys walk into a bar. The third guy ducks.


In [3]:
print("Number of records: ", jokes_df.shape[0])
print("Number of fields: ", jokes_df.shape[1])

Number of records:  100000
Number of fields:  2


In [4]:
jokes_df['Joke']


0        [me narrating a documentary about narrators] "...
1        Telling my daughter garlic is good for you. Go...
2        I've been going through a really rough period ...
3        If I could have dinner with anyone, dead or al...
4           Two guys walk into a bar. The third guy ducks.
                               ...                        
99995    Every time I walk into a singles bar I can hea...
99996    how wide is the universe? how long is a piece ...
99997    A man goes to a halloween party wearing nothin...
99998                           I don't Bolivia Peru-v it.
99999    What's the world's longest Ted Talk? How I Met...
Name: Joke, Length: 100000, dtype: object

In [5]:
# Function to clean text
def clean_text(text):

    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    return text

# Apply the cleaning function to the Joke column
jokes_df['Cleaned_Joke'] = jokes_df['Joke'].apply(clean_text)

jokes_df[['Joke', 'Cleaned_Joke']]


,Joke,Cleaned_Joke
0,"[me narrating a documentary about narrators] ""...",me narrating documentary about narrators can ...
1,Telling my daughter garlic is good for you. Go...,telling my daughter garlic is good for you goo...
2,I've been going through a really rough period ...,i ve been going through really rough period at...
3,"If I could have dinner with anyone, dead or al...",if could have dinner with anyone dead or alive...
4,Two guys walk into a bar. The third guy ducks.,two guys walk into bar the third guy ducks
...,...,...
99995,Every time I walk into a singles bar I can hea...,every time walk into singles bar can hear mom ...
99996,how wide is the universe? how long is a piece ...,how wide is the universe how long is piece of ...
99997,A man goes to a halloween party wearing nothin...,a man goes to halloween party wearing nothing ...
99998,I don't Bolivia Peru-v it.,i don bolivia peru it


In [6]:
#Tokenizer process
tokenizer = Tokenizer()
#fit
tokenizer.fit_on_texts(jokes_df['Joke'])
#assign length of word index
total_words = len(tokenizer.word_index) + 1
total_words

46923

In [7]:
#declaring ngrams
input_sequences = []
#spliting the sentence
for line in jokes_df['Joke']:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [8]:
input_sequences[:5]

[[14, 8701],
 [14, 8701, 1],
 [14, 8701, 1, 3039],
 [14, 8701, 1, 3039, 43],
 [14, 8701, 1, 3039, 43, 25110]]

In [9]:
max_len = max([len(x) for x in input_sequences])

In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen = max_len, padding='pre')

In [11]:
padded_input_sequences

array([[   0,    0,    0, ...,    0,   14, 8701],
       [   0,    0,    0, ...,   14, 8701,    1],
       [   0,    0,    0, ..., 8701,    1, 3039],
       ...,
       [   0,    0,    0, ...,   27,    3,  620],
       [   0,    0,    0, ...,    3,  620,   23],
       [   0,    0,    0, ...,  620,   23,  379]], dtype=int32)

In [12]:
X = padded_input_sequences[:,:-1]

In [13]:
y = padded_input_sequences[:,-1]

In [14]:
total_rows = 100000
total_words = 46923
batch_size = 1000 

In [15]:
def batch_generator(y, batch_size, total_words):
    num_batches = int(np.ceil(len(y) / batch_size))
    for i in range(num_batches):
        start = i * batch_size
        end = min(start + batch_size, len(y))  
        y_batch = y[start:end]
        y_batch_encoded = tf.keras.utils.to_categorical(y_batch, num_classes=total_words)
        yield y_batch_encoded


In [16]:
y = padded_input_sequences[:,-1]
# Using the generator to process data
for encoded_batch in batch_generator(y, batch_size, total_words):
    pass

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

total_words = 46923
  
max_sequence_len = 10  

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.build(input_shape=(None, max_sequence_len-1))

print(model.summary())



/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 9, 100)         │     4,692,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 150)            │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 46923)          │     7,085,373 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,928,273 (45.50 MB)

 Trainable params: 11,928,273 (45.50 MB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
from tensorflow.keras.utils import to_categorical
import numpy as np

# Convert labels to a sparse matrix format
y_sparse = np.array(y, dtype='int32')  # Ensure y is an integer array for sparse representation

# sparse categorical crossentropy loss to handle sparse labels
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit model with sparse labels
model.fit(X, y_sparse, epochs=50,  batch_size= 512)

Epoch 1/50
3243/3243 ━━━━━━━━━━━━━━━━━━━━ 1411s 435ms/step - accuracy: 0.0721 - loss: 6.9527
Epoch 2/50
3243/3243 ━━━━━━━━━━━━━━━━━━━━ 1478s 440ms/step - accuracy: 0.1607 - loss: 5.7324
Epoch 3/50
3243/3243 ━━━━━━━━━━━━━━━━━━━━ 1387s 428ms/step - accuracy: 0.2303 - loss: 4.6811
Epoch 7/50
3243/3243 ━━━━━━━━━━━━━━━━━━━━ 1392s 425ms/step - accuracy: 0.2417 - loss: 4.5312
Epoch 8/50
3243/3243 ━━━━━━━━━━━━━━━━━━━━ 1395s 423ms/step - accuracy: 0.2528 - loss: 4.4042
Epoch 9/50
3243/3243 ━━━━━━━━━━━━━━━━━━━━ 1410s 435ms/step - accuracy: 0.2635 - loss: 4.2874
Epoch 10/50
3243/3243 ━━━━━━━━━━━━━━━━━━━━ 1438s 427ms/step - accuracy: 0.2742 - loss: 4.1835
Epoch 11/50
3243/3243 ━━━━━━━━━━━━━━━━━━━━ 1380s 426ms/step - accuracy: 0.2836 - loss: 4.0947
Epoch 12/50
3243/3243 ━━━━━━━━━━━━━━━━━━━━ 1371s 416ms/step - accuracy: 0.2932 - loss: 4.0127
Epoch 13/50
3243/3243 ━━━━━━━━━━━━━━━━━━━━ 1401s 416ms/step - accuracy: 0.3019 - loss: 3.9382
Epoch 14/50
3243/3243 ━━━━━━━━━━━━━━━━━━━━ 1318s 406ms/step - accu